In [1]:
import torch
from oa.model import PaiNN

/home/xmcao/.conda/envs/install/envs/oa_reactdiff/lib/python3.10/site-packages/torch_geometric/typing.py:54: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: /lib64/libm.so.6: version `GLIBC_2.29' not found (required by /home/xmcao/.conda/envs/install/envs/oa_reactdiff/lib/python3.10/site-packages/libpyg.so)
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
/home/xmcao/.conda/envs/install/envs/oa_reactdiff/lib/python3.10/site-packages/torch_geometric/typing.py:110: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /lib64/libm.so.6: version `GLIBC_2.29' not found (required by /home/xmcao/.conda/envs/install/envs/oa_reactdiff/lib/python3.10/site-packages/libpyg.so)
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "


In [2]:
model = PaiNN(num_feat=128, out_channels=128, in_hidden_channels=8, cutoff=5.0, n_rbf=20, num_interactions=5)

In [3]:
s0 = torch.rand(9, 8, dtype=torch.float32)

In [4]:
from torch_geometric.nn import radius_graph
pos = torch.tensor([[ 0.0072, -0.5687,  0.0000],
        [-1.2854,  0.2499,  0.0000],
        [ 1.1304,  0.3147,  0.0000],
        [ 0.0392, -1.1972,  0.8900],
        [ 0.0392, -1.1972, -0.8900],
        [-1.3175,  0.8784,  0.8900],
        [-1.3175,  0.8784, -0.8900],
        [-2.1422, -0.4239,  0.0000],
        [ 1.9857, -0.1365,  0.0000]], dtype = torch.float)

# edge_attr: inter_atomic distances
edge_index = radius_graph(pos, r=1.70, batch=None, loop=False)

In [5]:
out = model(s0, pos, edge_index)

In [6]:
from e3nn import o3
rot = o3.rand_matrix()
pos_rot = pos @ rot
edge_index_rot = radius_graph(pos_rot, r=1.70, batch=None, loop=False)
out_rot = model(s0, pos_rot, edge_index_rot)

In [7]:
(out[1] @ rot - out_rot[1]).sum()

tensor(-4.8429e-08, grad_fn=<SumBackward0>)

In [8]:
out[1].shape

torch.Size([9, 3])

In [9]:
(out[0] - out_rot[0]).sum()

tensor(-3.5018e-07, grad_fn=<SumBackward0>)

In [14]:
out[1]

tensor([[ 7.2919e-03, -5.6931e-01, -3.9026e-05],
        [-1.2926e+00,  2.5303e-01, -8.8065e-04],
        [ 1.1394e+00,  3.0648e-01,  1.3142e-04],
        [ 3.9006e-02, -1.1940e+00,  8.8569e-01],
        [ 3.8886e-02, -1.1922e+00, -8.8277e-01],
        [-1.3173e+00,  8.7470e-01,  8.8491e-01],
        [-1.3173e+00,  8.7461e-01, -8.8471e-01],
        [-2.1386e+00, -4.2105e-01, -2.0991e-05],
        [ 1.9787e+00, -1.3279e-01, -2.9483e-06]], grad_fn=<AddBackward0>)

In [8]:
import numpy as np
from torch import Tensor
import torch
def compute_frag_index(atoms_mask_rtp: Tensor) -> np.ndarray:
    counts = [
        torch.where(atoms_mask_rtp == ii)[0].numel()
        for ii in torch.unique(atoms_mask_rtp)
    ]
    return np.concatenate([np.array([0]), np.cumsum(counts)])

In [14]:
atoms_mask_rtp = torch.tensor([0, 0, 0, 0, 1, 1, 2, 2, 2], dtype=torch.long)
compute_frag_index(atoms_mask_rtp)

array([0, 4, 6, 9])